# Run your first Structured Streaming workload
[docs](https://docs.databricks.com/aws/en/structured-streaming/tutorial)

## Use Auto Loader to read streaming data from object storage

load ข้อมูล json ด้วย auto loader ให้เปลี่ยนชื่อ checkpoint path เป็น volume หรือ path ที่เราสามารถใช้งานได้ ในที่นี้ใช้ vol, path ที่เคยสร้างไว้แล้ว

In [0]:
file_path = "/databricks-datasets/structured-streaming/events"
checkpoint_path = "/Volumes/workspace/default/tutorial/sscheckpoint/"

raw_df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", checkpoint_path)
    .load(file_path)
)

## Perform a streaming transformation

In [0]:
from pyspark.sql.functions import col, current_timestamp

transformed_df = (raw_df.select(
    "*",
    col("_metadata.file_path").alias("source_file"),
    current_timestamp().alias("processing_time")
    )
)

การ display df จากโค้ดข้างบนนี้ ตามโค้ด display ข้างล่างถ้ารันครั้งแรกแล้วจะรันผ่าน แต่รันครั้งต่อไปจะไม่ผ่าน เนื่องจากเป็นการเกี่ยวกับ [output mode](https://docs.databricks.com/aws/en/structured-streaming/output-mode) ซึ่ง display จะใช้ได้กับ complete mode แต่ df ของเราไม่ได้มี aggregate ใดๆ (เป็น append mode) ทำให้การ display ไม่สำเร็จ

มีวิธีแก้หลายแบบ เช่น การใช้ aggregation เพิ่มเข้าไปใน df, เปลี่ยน checkpoint
ในที่นี้จะใช้การลบ folder checkpoint เดิมด้วยโค้ดที่แปะไว้ด้านล่าง แล้วค่อยรัน cell ด้านบนใหม่อีกรอบ ต่อด้วย cell ที่ display ใหม่

In [0]:
dbutils.fs.rm("/Volumes/workspace/default/tutorial/sscheckpoint/", recurse=True)

In [0]:
display(
  transformed_df,
  checkpointLocation ="/Volumes/workspace/default/tutorial/sscheckpoint/"
)

## Perform an incremental batch write to Delta Lake

(ขออนุญาตย่อ Structured Streaming เป็น ss)

ตัวอย่างโค้ดนี้ จะเป็นการเขียนข้อมูลลง Delta Lake ซึ่งใช้ trigger แบบ availableNow เพราะการ trigger นี้ จะสั่งให้ตัว ss process record ทั้งหมดที่ยังไม่ได้ประมวลจาก source dataset แล้วค่อย shut down

การรันโค้ดนี้ทำให้เราไม่ต้องกังวลว่ามันจะไปกระทบอะไรกับการ streaming ที่ทำงานอยู่

> **สำคัญ** : ต้องระบุ checkpoint ที่ไม่ซ้ำเดิมสำหรับ streaming writer แต่ละอันที่เขียนด้วย เพราะ checkpoint จะให้ unique identity สำหรับ stream, track record ทั้งหมดที่เกี่ยวกับการ process อะไรต่างๆ และรวมไปถึงพวก state ที่มันเกี่ยวข้องกับ streaming

In [0]:
# ปรับชื่อ target path, checkpoint path ตามความเหมาะสม โดยถ้ายังไม่มี มันจะสร้าง path ตามที่เราระบุให้เอง

target_path = "/Volumes/workspace/default/tutorial/ss-tutorial/"
checkpoint_path = "/Volumes/workspace/default/tutorial/ss-checkpoint/"

transformed_df.writeStream.trigger(availableNow=True).option("checkpointLocation", checkpoint_path).option("path", target_path).start()

จาก docs tutorial เขาบอกเพิ่มมาด้วยว่า ในตัวอย่างนี้ เพราะมันไม่มี record ใหม่ๆ ถ้ารันใหม่ไปอีกก็จะไม่มีข้อมูลเพิ่มเข้ามา

และในแง่ของ production จริง ให้ระวังเรื่องการเสียค่าใช้จ่ายแบบที่เราไม่คาดคิดคาดฝัน เนื่องจาก ss ไม่ได้หยุดการทำงานแบบ auto ต่อให้ปิด compute ไป เพราะฉะนั้นอย่าลืมหยุด query ก่อนด้วย!

## Read data from Delta Lake, transform, and write to Delta Lake

delta lake รองรับกับ ss ทั้งในรูปแบบ source & sink โดยที่ตัวอย่างข้างล่างจะเป็นการอ่านข้อมูลจากตารางที่เป็น delta table แบบทีละน้อย มา join กับอีกตาราง(ที่เป็น delta เหมือนกัน) และเขียน delta table ใหม่อีกตาราง

(สามารถรันโค้ดด้านล่างได้ แต่ต้องเปลี่ยน path ของ checkpoint ให้เป็น path ใหม่, รวมถึงการใช้ชื่อตารางต่างๆ ด้วย ในที่นี้ใช้ตารางเก่าของ customers จากตัวอย่างของ LDP เนื่องจาก customers, customers_history_agg มาจากการรัน pipeline ในเรื่องก่อนหน้านี้ ซึ่งทำให้ตาราง 2 อันนี้มี id เดียวกัน และตารางบน databricks เก็บแบบ [delta table โดย default](https://docs.databricks.com/aws/en/delta/#getting-started-with-delta-lake) อยู่แล้ว ดังนั้นเลยดึงข้อมูลอันนี้มาใช้ได้แบบไม่มีปัญหาอะไร)

In [0]:
(spark.readStream
    .table("customers")
    .join(spark.read.table("customers_history_agg"), on="id", how="left")
    .writeStream
    .trigger(availableNow=True)
    .option("checkpointLocation", "/Volumes/workspace/default/tutorial/ss-checkpoint/delta")
    .toTable("testread_dt")
)

## Read data from Kafka, transform, and write to Kafka

Apache Kafka และระบบรับส่งข้อความ (messaging buses) อื่นๆ  ให้ latency ต่ำที่สุดสำหรับ dataset ขนาดมหึมา เราสามารถใช้ Databricks เพื่อแปลงข้อมูลที่ ingested จาก Kafka แล้วเขียนข้อมูลกลับไปยัง Kafka ได้

ตัวอย่าง code ที่ databricks ให้มา
```

(spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "<server:ip>")
    .option("subscribe", "<topic>")
    .option("startingOffsets", "latest")
    .load()
    .join(spark.read.table("<table-name>"), on="<id>", how="left")
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "<server:ip>")
    .option("topic", "<topic>")
    .option("checkpointLocation", "<checkpoint-path>")
    .start()
)

```

สำหรับโค้ดข้างบนนี้ สามารถใช้เป็นตัวอย่างการประยุกต์ใช้เมื่อต้องอ่าน เขียน จาก kafka ได้ แต่ว่า เพื่อให้รู้ว่ามันทำงานได้จริงหรือเปล่า เลยไปทดลองสมัคร [Oracle cloud (free tier)](https://www.oracle.com/asean/cloud/free) มา เพื่อลองใช้ kafka ดูจริงๆบน databricks

In [0]:
%sql
-- สร้างตารางมา 1 ตาราง เพื่อใช้ join กับข้อมูลที่มาจาก kafka

CREATE TABLE dim_users (
  id STRING,
  name STRING
);

INSERT INTO dim_users VALUES
('1', 'Alice'),
('2', 'Bob');

หลังจากเรา setup มหาศาลบน cloud ได้แล้ว เราก็ลองทดลองส่งข้อความแล้วให้โค้ดด้านล่างลองอ่านมาจาก kafka จริงๆดู

In [0]:
import uuid
checkpoint_path = f'/Volumes/workspace/default/tutorial/kafka_ckpt/{uuid.uuid4()}'
(
    spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "168.138.166.160:29092")
        .option("subscribe", "test-topic")
        .option("startingOffsets", "earliest")
        .load()
        .selectExpr(
            "CAST(key AS STRING) AS id",
            "CAST(value AS STRING) AS message"
        )
        .join(
            spark.read.table("dim_users"),
            on="id",
            how="left"
        )
        .writeStream
        .format("memory")
        .option("checkpointLocation", checkpoint_path)
        .option("queryName", "kafka_memory")
        .trigger(availableNow=True)
        .start()
)

ในส่วนนี้ จะเห็นข้อความที่เราพิมพ์ส่งจาก console ของ host 
_**ในขณะที่ query ด้านบน กำลังทำงานอยู่**_
> start query -> ในขณะที่ query กำลังรันอยู่ -> พิมพ์ข้อความบน console ของ host -> query รันเสร็จ -> ลองเช็คด้วย sql ข้างล่าง

(เพราะข้อจำกัดของ free edition ที่ trigger ตามที่เราต้องการไม่ได้ เลยต้องใช้วิธีการแบบ ความไวเป็นเรื่องของปีศาจ เพื่อทดลองส่วนนี้ดู)

ถ้าไม่อยากแข่งกับเวลา ใช้ `.option("startingOffsets", "latest")` เปลี่ยนเป็น `earliest` เพราะมันจะอ่านข้อมูลที่มีใน kafka แล้วก่อนที่จะเริ่ม query



In [0]:
%sql
SELECT * FROM kafka_memory;

---

# Structured Streaming patterns on Databricks

[docs](https://docs.databricks.com/aws/en/structured-streaming/examples)

ส่วนใหญ่จะเป็นตัวอย่างโค้ดสำหรับทำงานกับ ss เนื่องจากข้อจำกัดของ free edition ที่ใช้ serverless compute ทำให้ลงเครื่องมือเสริมใน compute ที่เรามีไม่ได้ และหลายๆอย่างในนี้ต้องใช้เครื่องมือเสริมต่างๆ (พวก lib, connector, etcฯ) ทำให้ไม่ได้ทดลองรันจริงๆ

## Write to Cassandra as a sink for Structured Streaming in Python

ss ทำงานกับ cassandra โดยใช้ [Spark Cassandra Connector](https://github.com/apache/cassandra-spark-connector) ซึ่งจะรองรับทั้ง RDD, API, DF รวมถึงรองรับการเขียนข้อมูลแบบ streaming โดยตรง

ตัวอย่างข้างล่างคือ show ว่าถ้าจะเชื่อมต่อกับ host ตั้งแต่ 1 host ขึ้นไปใน cassandra db จะเป็นประมาณไหน ซึ่งเราจะต้อง config พวก checkpoint location, ชื่อ keyspace, ชื่อตาราง

(โค้ดด้านล่างไม่ได้ลองรันจริง!)

In [0]:
spark.conf.set("spark.cassandra.connection.host", "host1,host2")

df.writeStream \
  .format("org.apache.spark.sql.cassandra") \
  .outputMode("append") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .option("keyspace", "keyspace_name") \
  .option("table", "table_name") \
  .start()

## Write to Azure Synapse Analytics using `foreachBatch()` in Python

นำ [foreachBatch()](https://docs.databricks.com/aws/en/structured-streaming/foreach) มาช่วยในการเขียนผลจากการ aggregation ไปยังปลายทาง

ปกติ pattern จะประมาณนี้ `streamingDF.writeStream.foreachBatch(...)` ซึ่งมันจะช่วยให้ใช้ batch function กับทุกๆ micro-batch ใน streaming query ได้ และรับ 2 parameters คือ
- df ที่มี output เป็น micro-batch
- unique id ของ micro-batch

ปล. จริงๆโค้ดนี้ หากเรามี account ที่ครอบคลุมถึงบริการดังกล่าวสามารถลองได้ แต่จากการสมัคร account student เพื่อทดลองใช้เครื่องมือ Azure Synapse Analytics พบว่าสิทธิ์ Student ไม่ครอบคลุมถึงตรงนี้ เลยลองเล่นจริงๆไม่ได้


In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

def writeToSQLWarehouse(df, epochId):
  df.write \
    .format("com.databricks.spark.sqldw") \
    .mode('overwrite') \
    .option("url", "jdbc:sqlserver://<the-rest-of-the-connection-string>") \
    .option("forward_spark_azure_storage_credentials", "true") \
    .option("dbtable", "my_table_in_dw_copy") \
    .option("tempdir", "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>") \
    .save()

spark.conf.set("spark.sql.shuffle.partitions", "1")

checkpoint_path = "/path/to/checkpoint" # เพิ่มอันนี้มา
query = (
  spark.readStream.format("rate").load()
    .selectExpr("value % 10 as key")
    .groupBy("key")
    .count()
    .toDF("key", "count")
    .writeStream
    .foreachBatch(writeToSQLWarehouse)
    .outputMode("update")
    .option("checkpointLocation", checkpoint_path) # เพิ่มมา
    .start()
    )

## Write to Amazon DynamoDB using foreach() in Scala and Python _(in this case we use Python only ka)_

ในเคสนี้ เราจะใช้ foreach() ซึ่งทำให้เขียน output จาก streaming query ไปที่ไหนก็ได้

แน่นอนว่า เราไม่ได้มี Amazon DynamoDB (ลองดูของ aws มาแล้วพบว่าบัญชีฟรีใช้ได้ 6 เดือน😭 อีกทั้ง เหมือนจะทดลองอะไรบางอย่างผิดพลาดไปนิดหน่อย เลยโดน suspended account ไปแล้ว) เลยจะเล่าแต่ละข้อแบบคร่าวๆแทนค่ะ

In [0]:
# Step 1 : define function เพื่อใช้สร้าง DynamoDB Table
table_name = "PythonForeachTest"

def get_dynamodb():
  import boto3

  access_key = "<access key>"
  secret_key = "<secret key>"
  region = "<region name>"
  return boto3.resource('dynamodb',
                 aws_access_key_id=access_key,
                 aws_secret_access_key=secret_key,
                 region_name=region)

def createTableIfNotExists():
    '''
    Create a DynamoDB table if it does not exist.
    This must be run on the Spark driver, and not inside foreach.
    '''
    dynamodb = get_dynamodb()

    existing_tables = dynamodb.meta.client.list_tables()['TableNames']
    if table_name not in existing_tables:
      print("Creating table %s" % table_name)
      table = dynamodb.create_table(
          TableName=table_name,
          KeySchema=[ { 'AttributeName': 'key', 'KeyType': 'HASH' } ],
          AttributeDefinitions=[ { 'AttributeName': 'key', 'AttributeType': 'S' } ],
          ProvisionedThroughput = { 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
      )

      print("Waiting for table to be ready")

table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

step 2 : define class & method ที่เขียนข้อมูลลงใน DynamoDB แล้วเรียกใช้จาก foreach ในที่นี้มี 2 วิธี คือใช้ function และใช้ class
- function : จะเป็นการเรียก function ต่อ 1 row เป็นแบบ simple

In [0]:
# Step 2 : Use a function
def sendToDynamoDB_simple(row):
  '''
  Function to send a row to DynamoDB.
  When used with `foreach`, this method is going to be called in the executor
  with the generated output rows.
  '''
  # Create client object in the executor,
  # do not use client objects created in the driver
  dynamodb = get_dynamodb()

  dynamodb.Table(table_name).put_item(
      Item = { 'key': str(row['key']), 'count': row['count'] })

- class (มี method : open, process, close) : จะมีหลายกระบวนการนิดนึง แต่ว่าเหมาะกับการทำงานจริงๆมากกว่า  เพราะเขียนได้หลาย row ต่อ 1 connection และพอเรียก connection คือเปิดแค่ครั้งเดียว

In [0]:
# Step 2 : Use a class
class SendToDynamoDB_ForeachWriter:
  '''
  Class to send a set of rows to DynamoDB.
  When used with `foreach`, copies of this class is going to be used to write
  multiple rows in the executor. See the python docs for `DataStreamWriter.foreach`
  for more details.
  '''

  def open(self, partition_id, epoch_id):
    # This is called first when preparing to send multiple rows.
    # Put all the initialization code inside open() so that a fresh
    # copy of this class is initialized in the executor where open()
    # will be called.
    self.dynamodb = get_dynamodb()
    return True

  def process(self, row):
    # This is called for each row after open() has been called.
    # This implementation sends one row at a time.
    # For further enhancements, contact the Spark+DynamoDB connector
    # team: https://github.com/audienceproject/spark-dynamodb
    self.dynamodb.Table(table_name).put_item(
        Item = { 'key': str(row['key']), 'count': row['count'] })

  def close(self, err):
    # This is called after all the rows have been processed.
    if err:
      raise err

In [0]:
# Step 3 : เรียกใช้ foreach ใน streaming query โดยใช้ object/function ที่ define ไป
from pyspark.sql.functions import *

spark.conf.set("spark.sql.shuffle.partitions", "1")

query = (
  spark.readStream.format("rate").load()
    .selectExpr("value % 10 as key")
    .groupBy("key")
    .count()
    .toDF("key", "count")
    .writeStream
    .foreach(SendToDynamoDB_ForeachWriter())
    #.foreach(sendToDynamoDB_simple)  // alternative, use one or the other
    .outputMode("update")
    .start()
)

## Stream-Stream joins

จะอยู่ในอีก [Notebook](url)